In [10]:
import uproot
import numpy as np #math and science package
import scipy as sp #math and science package
import awkward as ak #root files are usuallt awkward arrays 
import matplotlib.pyplot as plt #plot stuff
import itertools
from my_functions import *

#Open the file
file= uproot.open("/home/ruben/Escritorio/BachelorThesisRuben/Data/Muon_trees.root")

#Get both keys
MuonTree_Zmumu=file["MuonTree_Zmumu;1"]
MuonTree_ZeroBias=file["MuonTree_ZeroBias;1"] 

#Get eta, phi and pt
eta=MuonTree_Zmumu["muon_eta"].array()
phi=MuonTree_Zmumu["muon_phi"].array()
pt=MuonTree_Zmumu["muon_pt"].array()

#Let's use 0 quality muons only to reduce significantly the computation time. 
data1=MuonTree_Zmumu["muon_quality"].array() #save the quality vector
value=0 #set desired quality to 0

data2=MuonTree_Zmumu["muon_pt"].array()
pt=quality_selector(data1,data2,value)
data2=MuonTree_Zmumu["muon_eta"].array()
eta=quality_selector(data1,data2,value)
data2=MuonTree_Zmumu["muon_phi"].array()
phi=quality_selector(data1,data2,value)

Only 21.265 % of the data has been selected
Only 21.265 % of the data has been selected
Only 21.265 % of the data has been selected


In [11]:
#This generates 3 arrays very similar to:

# eta=MuonTree_Zmumu["muon_eta"].array()
# phi=MuonTree_Zmumu["muon_phi"].array()
# pt=MuonTree_Zmumu["muon_pt"].array()

#but that only contain pairs of data, because all events with more than two muons have been processed by pair_selector

pt, eta, phi = get_pair_variables(pt,eta,phi)

In [12]:
#With this function we get an array that contains the values of delta r for every event

dr=get_all_delta_r(eta,phi)

In [13]:
#Simple way to check how many event are below or above a certain threshold

indices = np.where(dr < 0.4)[0]